In [20]:
import pandas as pd

import re

pip install googlemaps

pip install simplejson

pip install --upgrade google-auth-oauthlib

import googlemaps as googlemaps

from pytz import timezone

from tqdm import tqdm

##My coordinates were contained in an excel named "Zip - Lat Loon.xlsx".
data = pd.read_excel("Zip - Lat Loon.xlsx")  
data['coordinates'] = data['LAT'].astype(str) + ", " +data['LNG'].astype(str)

print(data)

In [21]:
API_key = 'your google api Key goes Here'   #personal API Key

# I define the function that is going to calculate the distances with different methods.

def get_travel_data_gmaps(origin, destination, API_key):
    # Initialize the Google Maps client
    gmaps = googlemaps.Client(key=API_key)

    # Define the departure time in the preferred time zone. We will use New York time
    # We also define the departure time. It must be a future time.
    miami_timezone = timezone('America/New_York')
    departure_time = miami_timezone.localize(pd.Timestamp("2023-10-02 08:00:00")) 

    # Make the request for driving mode
    result_driving = gmaps.distance_matrix(
        origin,
        destination,
        mode="driving",
        traffic_model="best_guess",
        departure_time=departure_time,
        units="imperial",
        language="en-US"
    )

    # Make the request for transit mode
    result_transit = gmaps.distance_matrix(
        origin,
        destination,
        mode="transit",
        traffic_model="best_guess",
        departure_time=departure_time,
        units="imperial",
        language="en-US"
    )

    # Extract information for driving mode
    driving_distance = result_driving['rows'][0]['elements'][0].get('distance', {}).get('text', 'N/A')
    driving_duration = result_driving['rows'][0]['elements'][0].get('duration', {}).get('text', 'N/A')

    # Extract information for transit mode or set to 'N/A' if no transit route
    if result_transit['status'] == 'ZERO_RESULTS':
        transit_distance = 'N/A'
        transit_duration = 'N/A'
    else:
        transit_distance = result_transit['rows'][0]['elements'][0].get('distance', {}).get('text', 'N/A')
        transit_duration = result_transit['rows'][0]['elements'][0].get('duration', {}).get('text', 'N/A')

    # Create a DataFrame to store the results
    result_df = pd.DataFrame({
        "Mode": ["driving", "transit"],
        "Distance": [driving_distance, transit_distance],
        "Duration": [driving_duration, transit_duration],
        "Status": [result_driving.get('status', 'N/A'), result_transit.get('status', 'N/A')]
    })

    return result_df

,index,Work State,Home Postal,Home Country,LAT,LNG,coordinates
0,156,Florida,32003,United States of America,30.095584,-81.710162,"30.095584, -81.710162"
1,157,Florida,32024,United States of America,30.091953,-82.731070,"30.091953, -82.73107"
2,158,Florida,32034,United States of America,30.609692,-81.491720,"30.609692, -81.49172"
3,159,Florida,32065,United States of America,30.152238,-81.797016,"30.152238, -81.797016"
4,160,Florida,32066,United States of America,30.039765,-83.205802,"30.039765, -83.205802"


In [17]:
#You must define your destinations. They could more than one, in my case I needed to calculate
# the trip for the people for 2 different destinations, I created list.
destinations = [
   
    ["destination1", "27.200, -80.3222"],
    ["destination2", "31.22, -52.21"] #these are not real places
]

destinations_pd = pd.DataFrame(offices, columns=["destination name", "Office_Zip_Coord"])

# Initialize an empty DataFrame to store the results
result_df = pd.DataFrame(columns=["Zip Code", "Work State", "LAT", "LNG", "Travel Mode", "destination",
                                  "Distance", "Duration", "Status"])

# Iterate over each row in data_florida and each office
for _, data_row in tqdm(data.iterrows(), total=len(data_test), desc="Processing Rows"):
    origin = (data_row['LAT'], data_row['LNG'])
    zip_code = data_row['Home Postal']
    work_state = data_row['Work State']
    coordinates = data_row['coordinates']

    for _, destin_row in destinations_pd.iterrows():
        destination = tuple(map(float, office_row['Office_Zip_Coord'].split(', ')))
        destinations_pd = destin_row['Office_Name']

        # Call the function to get travel data for driving mode
        travel_data_results = get_travel_data_gmaps(origin, destination, API_key)


        # Append the results to the result_df
        for _, travel_row in travel_data_results.iterrows():
            result_df = result_df.append({
                "Zip Code": zip_code,
                "Work State": work_state,
                "LAT": coordinates.split(',')[0],
                "LNG": coordinates.split(',')[1],
                "Travel Mode": travel_row["Mode"],
                "destination": destination, 
                "Distance": travel_row["Distance"],
                "Duration": travel_row["Duration"],
                "Status": travel_row["Status"]
            }, ignore_index=True)

       

# Display the final result_df
print(result_df)

result_df

In [ ]:
# Testeamos la función con 5 casos primero
## Ahora, vamos a correrla con todos los datos de florida!


In [23]:
offices = [
    ["Assurant Campus", "25.606126, -80.343496"],
    ["Landing & Waterford", "25.7794, -80.299475"],
    ["Dural Square", "25.827855, -80.316183"]
]

offices_pd = pd.DataFrame(offices, columns=["Office_Name", "Office_Zip_Coord"])

# Initialize an empty DataFrame to store the results
result_df_final = pd.DataFrame(columns=["Zip Code", "Work State", "LAT", "LNG", "Travel Mode", "Office",
                                  "Distance", "Duration", "Status"])

# Iterate over each row in data_florida and each office
for _, data_row in tqdm(data_florida.iterrows(), total=len(data_test), desc="Processing Rows"):
    origin = (data_row['LAT'], data_row['LNG'])
    zip_code = data_row['Home Postal']
    work_state = data_row['Work State']
    coordinates = data_row['coordinates']

    for _, office_row in offices_pd.iterrows():
        destination = tuple(map(float, office_row['Office_Zip_Coord'].split(', ')))
        office_name = office_row['Office_Name']

        # Call the function to get travel data for driving mode
        travel_data_results = get_travel_data_gmaps(origin, destination, API_key)


        # Append the results to the result_df_final
        for _, travel_row in travel_data_results.iterrows():
            result_df_final = result_df_final.append({
                "Zip Code": zip_code,
                "Work State": work_state,
                "LAT": coordinates.split(',')[0],
                "LNG": coordinates.split(',')[1],
                "Travel Mode": travel_row["Mode"],
                "Office": office_name,  # Use the correct office_name variable
                "Distance": travel_row["Distance"],
                "Duration": travel_row["Duration"],
                "Status": travel_row["Status"]
            }, ignore_index=True)

       

# Display the final result_df
print(result_df_final)

Processing Rows: 394it [10:17,  1.57s/it]                     

     Zip Code Work State        LAT          LNG Travel Mode  \
0       32003    Florida  30.095584   -81.710162     driving   
1       32003    Florida  30.095584   -81.710162     transit   
2       32003    Florida  30.095584   -81.710162     driving   
3       32003    Florida  30.095584   -81.710162     transit   
4       32003    Florida  30.095584   -81.710162     driving   
...       ...        ...        ...          ...         ...   
2359    33313    Florida  26.151527   -80.227316     transit   
2360    33313    Florida  26.151527   -80.227316     driving   
2361    33313    Florida  26.151527   -80.227316     transit   
2362    33313    Florida  26.151527   -80.227316     driving   
2363    33313    Florida  26.151527   -80.227316     transit   

                   Office Distance          Duration Status  
0         Assurant Campus   368 mi   5 hours 23 mins     OK  
1         Assurant Campus   377 mi  10 hours 49 mins     OK  
2     Landing & Waterford   351 mi    5 hours

In [24]:
result_df_final

,Zip Code,Work State,LAT,LNG,Travel Mode,Office,Distance,Duration,Status
0,32003,Florida,30.095584,-81.710162,driving,Assurant Campus,368 mi,5 hours 23 mins,OK
1,32003,Florida,30.095584,-81.710162,transit,Assurant Campus,377 mi,10 hours 49 mins,OK
2,32003,Florida,30.095584,-81.710162,driving,Landing & Waterford,351 mi,5 hours 8 mins,OK
3,32003,Florida,30.095584,-81.710162,transit,Landing & Waterford,368 mi,10 hours 53 mins,OK
4,32003,Florida,30.095584,-81.710162,driving,Dural Square,350 mi,5 hours 7 mins,OK
...,...,...,...,...,...,...,...,...,...
2359,33313,Florida,26.151527,-80.227316,transit,Assurant Campus,56.4 mi,3 hours 22 mins,OK
2360,33313,Florida,26.151527,-80.227316,driving,Landing & Waterford,32.6 mi,39 mins,OK
2361,33313,Florida,26.151527,-80.227316,transit,Landing & Waterford,37.1 mi,3 hours 8 mins,OK
2362,33313,Florida,26.151527,-80.227316,driving,Dural Square,31.1 mi,37 mins,OK


In [25]:
excel_file_path = r'C:\youruser\importasanexcel.xlsx'

# Export the DataFrame to Excel
result_df_final.to_excel(excel_file_path, index=False)

print(f"DataFrame exported to {excel_file_path}")

DataFrame exported to C:\Users\Lisandro\Documents\Miami Offices\output.xlsx


In [1]:
#  ;)   #